##### Description: Scrape UNHCR API for data on Syrian refugees in a specific countries of asylum
##### Author: Matthew Albert

In [77]:
# Import necesssary packages and modules
import requests
import json
import copy
from dateutil.parser import isoparse
from maps_utilities import get_map_info
from wikidata_utilities import get_page_data, get_page_id, get_page_title, get_img_url

In [78]:
# Define specific parameters for the scrape

syria_iso = "SYR"                              # ISO code for Syria (country of origin)
endpoints = ["population", "asylum-decisions"] # API endpoints to scrape
countries = ["DEU", "TUR", "JOR"]              # ISO codes for countries of asylum
num_instances = 50                             # Num items to return
year_from = 2010                               # Fetch results starting from this year (inclusive)
year_to = 2023                                 # Fetch results ending at this year (inclusive)

country_data = []
json_file_path = "./models_data/country_db.json"

In [79]:
# Define JSON structure that will be used to store the scraped data
country_instance = {
  "country_name" : "",
  "id" : "",
  "attributes" : {
    "country_iso3" : "",
    "flag_url" : "",
    "map_info" : "",
    "capital" : "",
    "num_refugees" : 0,
    "num_asylum_decisions" : 0,
    "year_of_decisions" : 0,
    "num_recognized" : 0,
    "num_other" : 0,
    "num_apps_rejected" : 0,
    "num_closed" : 0
  }
}

In [80]:
# Miscellaneous functions

# Extract the current capital(s) of a country from a list of previous capital cities
def extract_curr_capital(capital_data):
  # Store capitals in set to avoid duplicates
  capitals = set()
  if len(capital_data) == 1:
    # Return, only one capital city in list
    capitals.add(get_page_title(capital_data[0]["mainsnak"]["datavalue"]["value"]["id"]))
    return capitals
  for capital_entry in capital_data:
    # Store property ID for the "end time" field
    end_time_property = "P582"
    if end_time_property not in capital_entry["qualifiers"]:
      # This is a current capital city, add to list
      capitals.add(get_page_title(capital_entry["mainsnak"]["datavalue"]["value"]["id"]))
  return list(capitals)

# Extract the current flag of a country from a list of previous flags
def extract_curr_flag(flag_data):
  if (len(flag_data) == 1):
    # Only one flag in list, return
    return flag_data[0]["mainsnak"]["datavalue"]["value"]
  # Try to sort list of json objects by the "start time" field (P580)
  start_time = "P580"
  try:
    # Sort objects by ISO 8601 formatted dates
    sorted_flag_data = sorted(flag_data, key = lambda i: isoparse(i["qualifiers"][start_time][0]["datavalue"]["value"]["time"][1:]), reverse=True)
    # Return flag at front of list (most recent flag)
    return sorted_flag_data[0]["mainsnak"]["datavalue"]["value"]
  except:
    # Failed to sort list, most likely problem with data format, return first flag in list
    return flag_data[0]["mainsnak"]["datavalue"]["value"]

# Function to scrape country data that is only available through Wikidata API
def get_wikidata_fields(country_name, country_data):
  # Retrieve page ID of country page
  page_id = get_page_id(country_name)
  # Retrieve Wikidata data for country page
  json_response = get_page_data(page_id)
  # Attempt to scrape capital city data
  try:
    capital_property = "P36"
    # Extract property info from JSON response
    capital_entries = json_response["entities"][page_id]["claims"][capital_property]
    capitals = extract_curr_capital(capital_entries)
    # Store list with capital city (or cities)
    country_data["attributes"]["capital"] = capitals
  except:
    print(f"Failed to retrieve capital city data for {country_name}")
  # Attempt to scrape country flag image url
  try:
    flag_property = "P41"
    # Get image file name from page data
    file_name = extract_curr_flag(json_response["entities"][page_id]["claims"][flag_property])
    # Get image URL from file name
    img_url = get_img_url(file_name)
    # Store image URL in instance data
    country_data["attributes"]["flag_url"] = img_url
  except:
    print(f"Failed to retrieve flag image url for {country_name}")
    
# Function to select the index with the most recent and populous asylum decisions data
def filter_asylum_data(asylum_data):
  selected_index = -1
  # Store the most recent year (that is available)
  recent_year = asylum_data[-1]["year"]
  # Since most recent data is stored at end of list, iterate in reverse
  for i in range(len(asylum_data) - 1, -1, -1):
    if asylum_data[i]["year"] == recent_year:
      # Store index if this item contains more asylum decisions than the previously stored one
      if asylum_data[i]["dec_total"] > asylum_data[selected_index]["dec_total"]:
        selected_index = i
    else:
      # No more items with the most recent year, exit loop
      break
  return selected_index


In [81]:
# Function to scrape data for a single country (using a country's ISO3 code)
def scrape_country_data(country_code):
  # Retrieve deep copy of country_instance dict to store country data
  data = copy.deepcopy(country_instance)
  data["id"] = data["attributes"]["country_iso3"] = country_code
  # Iterate through each endpoint to scrape necessary attributes
  for endpoint in endpoints:
    params = {
      "limit" : num_instances,
      "yearFrom" : year_from,
      "yearTo" : year_to,
      "coo" : syria_iso,
      "coa" : country_code,
      "cf_type" : "ISO"       # Search for countries using ISO3 country codes
    }
    # Make request to UNHCR API
    response = requests.get(f"https://api.unhcr.org/population/v1/{endpoint}/", params=params)
    if response.status_code == 200:
      # Success, store data in json format
      response_data = response.json()
      assert(len(response_data["items"]) > 0)
      if endpoint == "population":
        # Select item at end of list (b/c it is the most recent year)
        item_index = -1
        # Store population dataset specific attribute
        data["attributes"]["num_refugees"] = response_data["items"][item_index]["refugees"]
        data["country_name"] = response_data["items"][item_index]["coa_name"]
      else:
        # Select data item with most recent year and most asylum decisions
        item_index = filter_asylum_data(response_data["items"])
        # Store asylum decisions dataset specific attributes
        data["attributes"]["num_asylum_decisions"] = response_data["items"][item_index]["dec_total"]
        data["attributes"]["year_of_decisions"] = response_data["items"][item_index]["year"]
        data["attributes"]["num_recognized"] = response_data["items"][item_index]["dec_recognized"]
        data["attributes"]["num_apps_rejected"] = response_data["items"][item_index]["dec_rejected"]
        data["attributes"]["num_other"] = response_data["items"][item_index]["dec_other"]
        data["attributes"]["num_closed"] = response_data["items"][item_index]["dec_closed"]
    else:
      # Error, print status code
      print(f"Request Error: {response.status_code}")
      print(f"Request URL: {response.url}")
      print(f"Country: {country_code}")
      exit(-1)
  # Scrape remaining data fields (flag image url, capital city (or cities), map info)
  data["attributes"]["map_info"] = get_map_info(data["country_name"])
  get_wikidata_fields(data["country_name"], data)
  return data

{'country_name': 'Germany',
 'id': 'DEU',
 'attributes': {'country_iso3': 'DEU',
  'flag_url': 'https://upload.wikimedia.org/wikipedia/en/b/ba/Flag_of_Germany.svg',
  'map_info': {'address_components': [{'long_name': 'Germany',
     'short_name': 'DE',
     'types': ['country', 'political']}],
   'formatted_address': 'Germany',
   'geometry': {'bounds': {'northeast': {'lat': 55.0815, 'lng': 15.0418962},
     'southwest': {'lat': 47.2701115, 'lng': 5.8663425}},
    'location': {'lat': 51.165691, 'lng': 10.451526},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 55.0815, 'lng': 15.0418962},
     'southwest': {'lat': 47.2701115, 'lng': 5.8663425}}},
   'place_id': 'ChIJa76xwh5ymkcRW-WRjmtd6HU',
   'types': ['country', 'political']},
  'capital': ['Berlin'],
  'num_refugees': 522575,
  'num_asylum_decisions': 73332,
  'year_of_decisions': 2022,
  'num_recognized': 15078,
  'num_other': 51754,
  'num_apps_rejected': 34,
  'num_closed': 6466}}